## 0.Install

### install python library

In [ ]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph langchain-ollama langchain-huggingface beautifulsoup4 sentence-transformers langchain langchain-openai ipywidgets jupyter

restart the kernel

Download [Ollama](https://ollama.com/) and run it


### LangSmith(Optional)

In [1]:
import os

# 设置环境变量
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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" # Place your API Key
os.environ["LANGCHAIN_PROJECT"] = "RAG-Ollama"


## 1.Create LLM

In [2]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.2", base_url="http://localhost:11434")


In [3]:
# 使用 LLM 调用
messages = "我在測試專案"
# messages = [
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": "你好，請問你能幫我什麼？"}
# ]
response = llm.invoke(messages)

print(response)

 Sounds like you're working on a testing project! What kind of testing are you doing? Is it unit testing, integration testing, or something else? Do you need help with any specific test cases or scenarios? I'm here to assist you.


## 2.Create Embeddings model

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

# 使用 Hugging Face 的嵌入模型
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [5]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

## 3.Load Doc

In [6]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

USER_AGENT environment variable not set, consider setting it to identify your requests.


## 4. Ask question

In [7]:
response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])

Task Decomposition is a technique that breaks down complex tasks into smaller and simpler steps by instructing a model to "think step by step". This technique uses Chain of Thought (CoT) or Tree of Thoughts to decompose big tasks into multiple manageable subgoals. It helps transform complicated tasks into more testable and interpretable parts, allowing the model to generate solutions step-by-step.
